<h1> Churn Analysis </h1>

In [1]:
!pip install keras

  Using cached Keras-2.1.4-py2.py3-none-any.whl


In [2]:
import datalab.bigquery as bq
import seaborn as sns
import pandas as pd
import numpy as np
import shutil
import os
import keras
import tensorflow as tf
print tf.__version__

/usr/local/envs/py2env/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


1.5.0


In [3]:
%sql --module letslookatthedata
SELECT voice, intplan, nummailmes, tdmin, tdcal, tdchar, temin, tecal, tecahr, tnmin, tncal, tnchar, timin, tical, ticharncsc, totalcal, label_neu FROM [go-de-internal:democases_go_reply_internal.churn_customer_preprocessed]

In [4]:
churn = bq.Query(letslookatthedata).to_dataframe()
churn.describe()

,nummailmes,tdmin,tdcal,tdchar,temin,tecal,tecahr,tnmin,tncal,tnchar,timin,tical,ticharncsc,totalcal,label_neu
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,7.755200,180.288900,100.029400,30.649668,200.636560,100.191000,17.054322,200.391620,99.919200,9.017732,10.261780,4.435200,2.771196,1.570400,0.141600
std,13.546393,53.894699,19.831197,9.162069,50.551309,19.826496,4.296843,50.527789,19.958686,2.273763,2.761396,2.456788,0.745514,1.306363,0.348674
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,143.700000,87.000000,24.430000,166.375000,87.000000,14.140000,166.900000,87.000000,7.510000,8.500000,3.000000,2.300000,1.000000,0.000000
50%,0.000000,180.100000,100.000000,30.620000,201.000000,100.000000,17.090000,200.400000,100.000000,9.020000,10.300000,4.000000,2.780000,1.000000,0.000000
75%,17.000000,216.200000,113.000000,36.750000,234.100000,114.000000,19.900000,234.700000,113.000000,10.560000,12.000000,6.000000,3.240000,2.000000,0.000000
max,52.000000,351.500000,165.000000,59.760000,363.700000,170.000000,30.910000,395.000000,175.000000,17.770000,20.000000,20.000000,5.400000,9.000000,1.000000


In [5]:
shuffled = churn.sample(frac=1)
trainsize = int(len(shuffled['label_neu']) * 0.70)
validsize = int(len(shuffled['label_neu']) * 0.15)

df_train = shuffled.iloc[:trainsize, :]
df_valid = shuffled.iloc[trainsize:(trainsize+validsize), :]
df_test = shuffled.iloc[(trainsize+validsize):, :]

In [6]:
def to_csv(df, filename):
  outdf = df.copy(deep=False)
  
  # reorder columns so that target is first column
  cols = outdf.columns.tolist()
  cols.remove('label_neu')
  cols.insert(0, 'label_neu')
  print (cols)  # new order of columns
  outdf = outdf[cols]
  outdf.to_csv(filename, header=False, index_label=False, index=False)
  
  print outdf.shape

to_csv(df_train, 'churn-train.csv')
to_csv(df_valid, 'churn-valid.csv')
to_csv(df_test, 'churn-test.csv')

['label_neu', u'voice', u'intplan', u'nummailmes', u'tdmin', u'tdcal', u'tdchar', u'temin', u'tecal', u'tecahr', u'tnmin', u'tncal', u'tnchar', u'timin', u'tical', u'ticharncsc', u'totalcal']
(3500, 17)
['label_neu', u'voice', u'intplan', u'nummailmes', u'tdmin', u'tdcal', u'tdchar', u'temin', u'tecal', u'tecahr', u'tnmin', u'tncal', u'tnchar', u'timin', u'tical', u'ticharncsc', u'totalcal']
(750, 17)
['label_neu', u'voice', u'intplan', u'nummailmes', u'tdmin', u'tdcal', u'tdchar', u'temin', u'tecal', u'tecahr', u'tnmin', u'tncal', u'tnchar', u'timin', u'tical', u'ticharncsc', u'totalcal']
(750, 17)


In [7]:
#baselinemodel...

In [8]:
PROJECT = 'go-de-internal'
BUCKET = 'storage-democases-go-reply-internal'
REGION = 'us-central1'
REPO = "/content/datalab/democases-go-reply-internal/churn-prediction"
os.listdir(REPO)

['yiannis_cloud_ml_in_datalab.ipynb',
 'Final Demo.ipynb',
 'Churn-with-Neural-Net.ipynb',
 'Untitled Notebook.ipynb',
 'testing-cloud-ml.ipynb',
 'testing-cloud-ml-checkpoint.ipynb',
 '.ipynb_checkpoints',
 'Churn-Prediction-with-LSTN.ipynb',
 'mnist_keras_ml_engine+%282%29.ipynb',
 'hyperparmTrainingOnNumberDetection.ipynb',
 'churn-train.csv',
 'churn-valid.csv',
 'running+Cloud+ML.ipynb',
 'churn-test.csv',
 'churn_with_neural_net',
 'hyperparameterTuning.ipynb',
 'create_churn_timedependent_dataset (1).ipynb']

In [9]:
# for bash
os.environ['PROJECT'] = PROJECT
os.environ['BUCKET'] = BUCKET
os.environ['REGION'] = REGION
os.environ['REPO'] = REPO

In [10]:
%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

Updated property [core/project].
Updated property [compute/region].


In [11]:
%bash
PROJECT_ID=$PROJECT
AUTH_TOKEN=$(gcloud auth print-access-token)
SVC_ACCOUNT=$(curl -X GET -H "Content-Type: application/json" \
    -H "Authorization: Bearer $AUTH_TOKEN" \
    https://ml.googleapis.com/v1/projects/${PROJECT_ID}:getConfig \
    | python -c "import json; import sys; response = json.load(sys.stdin); \
    print response['serviceAccount']")

echo "Authorizing the Cloud ML Service account $SVC_ACCOUNT to access files in $BUCKET"
gsutil -m defacl ch -u $SVC_ACCOUNT:R gs://$BUCKET
gsutil -m acl ch -u $SVC_ACCOUNT:R -r gs://$BUCKET  # error message (if bucket is empty) can be ignored
gsutil -m acl ch -u $SVC_ACCOUNT:W gs://$BUCKET

Authorizing the Cloud ML Service account service-462605511119@cloud-ml.google.com.iam.gserviceaccount.com to access files in storage-democases-go-reply-internal


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   133    0   133    0     0    339      0 --:--:-- --:--:-- --:--:--   340
No changes to gs://storage-democases-go-reply-internal/
No changes to gs://storage-democases-go-reply-internal/adult.data.csv
No changes to gs://storage-democases-go-reply-internal/churn_cloud/churn-test.csv
No changes to gs://storage-democases-go-reply-internal/churn_cloud/churn-train.csv
No changes to gs://storage-democases-go-reply-internal/churn_cloud/churn-valid.csv
No changes to gs://storage-democases-go-reply-internal/churn_model_180220_175843/e2b80417fe931b7ed44b08d1374eea2bcb1f7898472ef0c5e51373d725cf419e/example5-0.1.tar.gz
No changes to gs://storage-democases-go-reply-internal/churn_model_180220_175853/e171b54ea317d121eebaa18cfa3a24d49d32790c95e6e87579f7f1e87698108e/example5-0.1.tar.gz
No changes to gs://storage-democases-go-reply-internal/ch

In [43]:
%bash
echo $BUCKET
gsutil -m rm -rf gs://${BUCKET}/churn_with_neural_net/
gsutil -m cp ${REPO}/*.csv gs://${BUCKET}/churn_with_neural_net/

storage-democases-go-reply-internal


Removing gs://storage-democases-go-reply-internal/churn_with_neural_net/churn-test.csv#1519382676454587...
Removing gs://storage-democases-go-reply-internal/churn_with_neural_net/churn-train.csv#1519382676520068...
Removing gs://storage-democases-go-reply-internal/churn_with_neural_net/churn-valid.csv#1519382676498775...
Removing gs://storage-democases-go-reply-internal/churn_with_neural_net/churn_time_dependent_sorted.csv#1519382677065084...
Removing gs://storage-democases-go-reply-internal/churn_with_neural_net/churn_time_dependent_unsorted.csv#1519382677044396...
/ [5/5 objects] 100% Done                                                       
Operation completed over 5 objects.                                              
Copying file:///content/datalab/democases-go-reply-internal/churn-prediction/churn-test.csv [Content-Type=text/csv]...
Copying file:///content/datalab/democases-go-reply-internal/churn-prediction/churn_time_dependent_sorted.csv [Content-Type=text/csv]...
Copying f

In [ ]:
%bash
OUTDIR=gs://${BUCKET}/churn_with_neural_net_output/churn_trained
rm -rf vehicles.tar.gz churn_trained
gcloud ml-engine local train \
   --module-name=trainer.task \
   --package-path=${REPO}/churn_with_neural_net/trainer \
   --job-dir $OUTDIR \
   -- \
   --train_files=${REPO}/churn-train.csv \
   --eval_files=${REPO}/churn-valid.csv  \
   --num_epochs=10 \
   --output_dir=${REPO}/churn_trained

In [34]:
%bash
echo $BUCKET
gsutil -m rm -rf gs://${BUCKET}/churn_cloud
gsutil -m cp ${REPO}/*.csv gs://${BUCKET}/churn_cloud/

storage-democases-go-reply-internal


CommandException: 1 files/objects could not be removed.
Copying file:///content/datalab/democases-go-reply-internal/churn-prediction/churn-test.csv [Content-Type=text/csv]...
Copying file:///content/datalab/democases-go-reply-internal/churn-prediction/churn_time_dependent_sorted.csv [Content-Type=text/csv]...
Copying file:///content/datalab/democases-go-reply-internal/churn-prediction/churn_time_dependent_unsorted.csv [Content-Type=text/csv]...
Copying file:///content/datalab/democases-go-reply-internal/churn-prediction/churn-train.csv [Content-Type=text/csv]...
Copying file:///content/datalab/democases-go-reply-internal/churn-prediction/churn-valid.csv [Content-Type=text/csv]...
- [5/5 files][ 29.2 MiB/ 29.2 MiB] 100% Done                                    
Operation completed over 5 objects/29.2 MiB.                                     


In [32]:
%%bash
OUTDIR=gs://${BUCKET}/churn_cloud_output/churn_trained
JOBNAME=churn_model_$(date -u +%y%m%d_%H%M%S)
echo $OUTDIR $REGION $JOBNAME
gsutil -m rm -rf $OUTDIR
gcloud ml-engine jobs submit training $JOBNAME \
  --region=$REGION \
  --module-name=trainer.task \
  --package-path=${REPO}/churn_with_neural_net/trainer \
  --job-dir=$OUTDIR \
  --staging-bucket=gs://$BUCKET \
  --scale-tier=BASIC \
  --runtime-version=1.0 \
  -- \
  --train_files="gs://${BUCKET}/churn_cloud/churn-train.csv" \
  --eval_files="gs://${BUCKET}/churn_cloud/churn-valid.csv"  \
  --output_dir=$OUTDIR \
  --num_epochs=100

gs://storage-democases-go-reply-internal/churn_cloud/churn_trained us-central1 churn_model_180223_100803
jobId: churn_model_180223_100803
state: QUEUED


CommandException: 1 files/objects could not be removed.
/tools/google-cloud-sdk/lib/googlecloudsdk/core/util/files.py:622: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  for chunk in iter(lambda: fp.read(4096), ''):
Job [churn_model_180223_100803] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ml-engine jobs describe churn_model_180223_100803

or continue streaming the logs with the command

  $ gcloud ml-engine jobs stream-logs churn_model_180223_100803
